In [1]:
import sys
import neal
import greedy
import tabu
import time
import numpy as np

from pathlib import Path
current_path = Path().resolve()
sys.path.append(str(current_path / '../code/'))
from experiment import Experiment
from table import Table
from visualisation import Figure

# Load the data

In [2]:
data_folder = '../data/Travelling Salesman Problem/Small/'

file_names = ['bayg29.npz', 'bays29.npz', 'dantzig42.npz',
             'fri26.npz', 'gr17.npz', 'gr21.npz', 'gr24.npz']

loaded_files = [np.load(data_folder + file) for file in file_names]

obj_qubos = [i['cost_function_qubo'] for i in loaded_files]
obj_constants = [i['cost_function_constant'] for i in loaded_files]
con_qubos = [i['constraint_function_qubo'] for i in loaded_files]
con_constants = [i['constraint_function_constant'] for i in loaded_files]

# Prepare the data

Here we already get the qubos and constraints. So we only need to calculate penalties and get the full QUBOs.

In [3]:
minimisation = True # This is a minimisation problem
QUBOs, penalties = Experiment.data_prep_light(obj_qubos, con_qubos, 'Expected Constraint', minimisation)
qubo_sizes = [max(qubo, key=tuple)[0] + 1 for qubo in QUBOs]

# Run experiments

In [4]:
# The number of samples we want the sampler to return
repeats = 20

## Greedy

In [5]:
greedy_sampler = greedy.SteepestDescentSampler()
greedy_runs = Experiment.run_sampler(QUBOs, obj_qubos, obj_constants, con_qubos, con_constants, 
                                     greedy_sampler, repeats, num_reads=180)

100.0 %


## Simulated Annealing

In [6]:
sa_sampler = neal.SimulatedAnnealingSampler()
sa_runs = Experiment.run_sampler(QUBOs, obj_qubos, obj_constants, con_qubos, con_constants, 
                                 sa_sampler, repeats, num_reads=30)

100.0 %


## Tabu

In [7]:
tabu_sampler = tabu.TabuSampler()
tabu_runs = Experiment.run_sampler(QUBOs, obj_qubos, obj_constants, con_qubos, con_constants, 
                                   tabu_sampler, repeats, timeout=4000)

100.0 %


# Record the results

In [8]:
greedy_results = Table.record_results(greedy_runs, qubo_sizes, penalties, repeats, minimisation)
sa_results = Table.record_results(sa_runs, qubo_sizes, penalties, repeats, minimisation)
tabu_results = Table.record_results(tabu_runs, qubo_sizes, penalties, repeats, minimisation)

# Display the first repetition table
rep = 0
Table.display_side_by_side(greedy_results[rep], sa_results[rep], tabu_results[rep], titles=['Greedy', 'SA', 'Tabu'])

,Size,Penalty,Objective Function,Broken Constraints,Energy (minimisation)
0,784,238.0,2160,0,2160.0
1,784,326.0,2760,0,2760.0
2,1681,127.0,1184,0,1184.0
3,625,206.0,1298,0,1298.0
4,256,548.0,2390,0,2390.0
5,400,603.0,3504,0,3504.0
6,529,242.0,1773,0,1773.0
,Size,Penalty,Objective Function,Broken Constraints,Energy (minimisation)
0,784,238.0,2527,0,2527.0
1,784,326.0,3186,0,3186.0


# Explore the results

In [9]:
# Show total energies of all tries in all problems in a single df
energies_greedy = Table.columns_to_table(greedy_results, 'Energy (minimisation)')
energies_sa = Table.columns_to_table(sa_results, 'Energy (minimisation)')
energies_tabu = Table.columns_to_table(tabu_results, 'Energy (minimisation)')

energies_tabu

,Energy (minimisation) 0,Energy (minimisation) 1,Energy (minimisation) 2,Energy (minimisation) 3,Energy (minimisation) 4,Energy (minimisation) 5,Energy (minimisation) 6,Energy (minimisation) 7,Energy (minimisation) 8,Energy (minimisation) 9,Energy (minimisation) 10,Energy (minimisation) 11,Energy (minimisation) 12,Energy (minimisation) 13,Energy (minimisation) 14,Energy (minimisation) 15,Energy (minimisation) 16,Energy (minimisation) 17,Energy (minimisation) 18,Energy (minimisation) 19
0,1928.0,1931.0,1919.0,1873.0,1855.0,1848.0,1779.0,1833.0,1813.0,1777.0,1877.0,1858.0,1791.0,1867.0,1886.0,1851.0,1920.0,1821.0,1842.0,1741.0
1,2303.0,2303.0,2194.0,2230.0,2190.0,2283.0,2289.0,2170.0,2151.0,2258.0,2283.0,2270.0,2210.0,2215.0,2214.0,2337.0,2220.0,2259.0,2241.0,2217.0
2,756.0,699.0,767.0,704.0,775.0,699.0,699.0,699.0,699.0,750.0,699.0,704.0,699.0,767.0,704.0,699.0,750.0,699.0,782.0,704.0
3,1055.0,1029.0,1047.0,1068.0,1011.0,1030.0,1040.0,1092.0,1020.0,1059.0,1092.0,1093.0,1029.0,999.0,1036.0,1036.0,1048.0,995.0,1057.0,1027.0
4,2174.0,2160.0,2158.0,2118.0,2160.0,2088.0,2143.0,2167.0,2177.0,2181.0,2170.0,2195.0,2100.0,2135.0,2191.0,2200.0,2217.0,2103.0,2200.0,2162.0
5,3087.0,3076.0,2864.0,2898.0,2950.0,3026.0,2803.0,3107.0,3011.0,3120.0,2997.0,2927.0,2958.0,3104.0,3045.0,3005.0,2707.0,2969.0,3030.0,2945.0
6,1414.0,1392.0,1362.0,1435.0,1439.0,1411.0,1366.0,1403.0,1331.0,1373.0,1349.0,1373.0,1390.0,1403.0,1361.0,1441.0,1402.0,1406.0,1403.0,1371.0


In [10]:
# Show number of broken constraints of all tries in all problems in a single df
broken_constraints_greedy = Table.columns_to_table(greedy_results, 'Broken Constraints')
broken_constraints_sa = Table.columns_to_table(sa_results, 'Broken Constraints')
broken_constraints_tabu = Table.columns_to_table(tabu_results, 'Broken Constraints')

broken_constraints_greedy

,Broken Constraints 0,Broken Constraints 1,Broken Constraints 2,Broken Constraints 3,Broken Constraints 4,Broken Constraints 5,Broken Constraints 6,Broken Constraints 7,Broken Constraints 8,Broken Constraints 9,Broken Constraints 10,Broken Constraints 11,Broken Constraints 12,Broken Constraints 13,Broken Constraints 14,Broken Constraints 15,Broken Constraints 16,Broken Constraints 17,Broken Constraints 18,Broken Constraints 19
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [11]:
# Show feasible solutions
feasible_full_greedy = Table.feasibility_table(greedy_results)
feasible_full_sa = Table.feasibility_table(sa_results)
feasible_full_tabu = Table.feasibility_table(tabu_results)

feasible_full_tabu

,Feasible 0,Feasible 1,Feasible 2,Feasible 3,Feasible 4,Feasible 5,Feasible 6,Feasible 7,Feasible 8,Feasible 9,Feasible 10,Feasible 11,Feasible 12,Feasible 13,Feasible 14,Feasible 15,Feasible 16,Feasible 17,Feasible 18,Feasible 19
0,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True
1,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True
2,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True
3,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True
4,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True
5,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True
6,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True,True


This is the final and the most important table. It displays the number of feasible solutions achieved for every problem, the feasibility rate (feasible_soultions/total_solutions), the mean energy of the solutions and the standard deviation.

In [12]:
# Calculate number of feasible solutions with mean and SD (in all runs)
feasible_greedy = Table.feasibility_statistic(greedy_results)
feasible_sa = Table.feasibility_statistic(sa_results)
feasible_tabu = Table.feasibility_statistic(tabu_results)

# Display the table
Table.display_side_by_side(feasible_greedy, feasible_sa, feasible_tabu, titles=['Greedy', 'SA', 'Tabu'])

,Feasible,Feasibility rate,Energy mean,Energy SD
0,20.0,1.0,2155.600000,83.550426
1,20.0,1.0,2735.900000,94.120248
2,20.0,1.0,1152.500000,54.904223
3,20.0,1.0,1266.450000,37.038351
4,20.0,1.0,2407.750000,78.156304
5,20.0,1.0,3534.400000,211.806366
6,20.0,1.0,1664.050000,57.021210
Total,140.0,7.0,14916.650000,616.597129
Mean,20.0,1.0,2130.950000,88.085304
SD,0.0,0.0,850.057109,57.928441


# Save results

In [14]:
data_folder = '../Data/Produced/Travelling Salesman Problem/Small/'
broken_constraints_greedy.to_pickle(data_folder + 'expected_tsp_greedy_broken_constraints.pkl')
broken_constraints_sa.to_pickle(data_folder + 'expected_tsp_sa_broken_constraints.pkl')
broken_constraints_tabu.to_pickle(data_folder + 'expected_tsp_tabu_broken_constraints.pkl')

feasible_greedy.to_pickle(data_folder + 'expected_tsp_greedy_feasible.pkl')
feasible_sa.to_pickle(data_folder + 'expected_tsp_sa_feasible.pkl')
feasible_tabu.to_pickle(data_folder + 'expected_tsp_tabu_feasible.pkl')